## Interactuando con Github

In [13]:
from github import Github
import os
from dotenv import load_dotenv, find_dotenv
from time import sleep
import re
import unicodedata
import json
import pickle

load_dotenv(find_dotenv())


git_key = os.getenv("GITHUB_ADMIN_TOKEN")
g = Github(git_key)

def download_github_data():
    with open("github_admin/files/github_books.pkl", "wb") as file:
        books = _get_books()
        pickle.dump(books, file)


def load_github_data():
    books = []
    with open("github_admin/files/github_books.pkl", "rb") as file:
        try:
            while True:
                books.append(pickle.load(file))
        except EOFError:
            return books[0]
        
def _get_books():
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Devuelve:
        listado de gitbook de la organización CATEDU.
    """
    global g
    repos = list(g.get_organization("catedu").get_repos())
    gitbooks = [repo for repo in repos if _has_summary(repo)]
    return gitbooks

def _has_summary(repo):
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Parámetros:
        repo (Objeto PyGithub que representa un repositorio)

    Devuelve:
        True or False (Boolean) indicando si tiene o no SUMMARY.md
    """
    try:
        contents = [content.name for content in repo.get_contents("")]
        it_has_summary = "SUMMARY.md" in contents
        return it_has_summary
    except:
        print(f"######### El repo {repo.name} falla #############")
        
def repos_data():
    books = load_github_data()
    data = [
        {
            "repo_name": book.name,
            "master_url": book.html_url,
            "gh-pages": f"https://catedu.github.io/{book.name}/",
        }
        for book in books
    ]
    return data

In [12]:
download_github_data()

In [16]:
books = load_github_data()

In [21]:
contents = books[0].get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(books[0].get_contents(file_content.path))
    else:
        print(file_content)

ContentFile(path=".gitignore")
ContentFile(path=".nojekyll")
ContentFile(path="FOOTER.md")
ContentFile(path="LICENSE")
ContentFile(path="README.md")
ContentFile(path="SUMMARY.md")
ContentFile(path="book.json")
ContentFile(path="consultar_catlogos_de_otras_bibliotecas.md")
ContentFile(path="consultar_el_catlogo_de_fondos_de_la_biblioteca_de_mi_centro.md")
ContentFile(path="crditos.md")
ContentFile(path="darme_de_alta_en_abiesweb_para_usar_la_biblioteca_de_mi_centro_escolar.md")
ContentFile(path="escribir_una_opinin_sobre_algn_fondo_de_los_que_hemos_tomado_en_prstamo.md")
ContentFile(path="modificar_mi_contrasea_de_acceso.md")
ContentFile(path="realizar_reservas_de_ejemplares.md")
ContentFile(path="seguir_las_noticias_de_la_biblioteca_de_mi_centro.md")
ContentFile(path="solicitar_una_prrroga_del_prstamo_actual.md")
ContentFile(path="img/Boletin.JPG")
ContentFile(path="img/BusquedaEjemplaresLEctor.JPG")
ContentFile(path="img/BusquedaEnRed.JPG")
ContentFile(path="img/ListadoCatalogoLector.

## Interactuando con SpreadSheets de google

In [49]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pandas as pd

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1u-OHCrt3TQdaI8l2K2_xvzum8ALIbip0r0_QSxX6H4E'
SAMPLE_RANGE_NAME = 'Objetivos!A1:C122'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    
    return pd.DataFrame(values)

In [50]:
main()

,0,1,2
0,Codigo,Curso,Objetivos
1,10,ABP APRENDIZAJE BASADO EN PROYECTOS INFANTIL P...,• Analizar los principios de la enseñanza basa...
2,20,ABP APRENDIZAJE BASADO EN PROYECTOS SECUNDARIA...,• Analizar los principios de la enseñanza basa...
3,25,AJEDREZ EN LA ESCUELA,None
4,40,ALIMENTACIÓN Y NUTRICIÓN PARA UNA VIDA SALUDABLE,• Contribuir a la prevención y control de los ...
...,...,...,...
117,3.040,INTERSECCIONALIDAD. CONSTRUCCIÓN DE IDENTIDADE...,None
118,3.010,CONVIVENCIA,None
119,3.100,TRABAJO FIN DE ITINERAIRO IGUALDAD,None
120,770,TRATAMIENTO DIGITAL DE LA IMAGEN PARA LA ENSEÑ...,• Conocer los fundamentos básicos de la Imagen...
